# Problem Statement:

 A company has thirteen existing factories with 600 to 3500 units of supply. Four of these factories will be closed and new factories with 5000 units of supply will be located in 2 of their 148 customer sites. Which existing factories should be closed, and which new sites should be selected to minimize total shipping costs if customer demand cannot be split between factories? 

In [1]:
import pandas as pd
import numpy as np

In [2]:
supply=pd.read_csv(r"C:\Users\Admin\Desktop\Meet_M2_AOSC\QFolder\Factories.csv")
demand=pd.read_excel(r"C:\Users\Admin\Desktop\Meet_M2_AOSC\QFolder\Customers.xls")
location=pd.read_csv(r"C:\Users\Admin\Desktop\Meet_M2_AOSC\QFolder\Location_data.csv")

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


Problem consitute of 13 supply locations and 148 demand locations for the oil requirement throughout the globe. As unit cost for transportation is not mentioned in problem we have to assume 1 unit cost per unit area.

In [3]:
supply.shape, demand.shape, location.shape

((13, 2), (148, 2), (161, 3))

cost matrix need to be computed using locations provided.

In [4]:
dist=np.zeros([13,148])

In [5]:
location=location.set_index('Location')
location.head()

,LAT,LONG
Location,,
"Birmingham, AL",33.524755,86.812740
"Mobile, AL",30.679523,88.103280
"Montgomery, AL",32.361538,86.279118
"Tuscaloosa, AL",33.206540,87.534607
"Anchorage, AK",61.191900,149.762097


In [6]:
supply.head()

,Location,Supply
0,"Tucson, AZ",2000
1,"Fresno, CA",2200
2,"Lansing, MI",2600
3,"Bozeman, MT",600
4,"Reno, NV",2500


In [7]:
demand1=demand.T
demand1.head()

,0,1,2,3,4,5,6,7,8,9,...,138,139,140,141,142,143,144,145,146,147
Location,"Birmingham, AL","Mobile, AL","Montgomery, AL","Tuscaloosa, AL","Anchorage, AK","Fairbanks, AK","Juneau, AK","Flagstaff, AZ","Phoenix, AZ","Scottsdale, AZ",...,"Wheeling, WV","Green Bay, WI","Madison, WI","Milwaukee, WI","Oshkosh, WI","Casper, WY","Cheyenne, WY","Cody, WY","Evanston, WY","Laramie, WY"
Demand,200,100,150,50,50,100,200,50,300,100,...,150,150,250,300,50,50,150,50,150,100


## Calculating Cost matrix

For every demand supply pair we have a unique cost associated which will help the transportation problem to minimize the cost according to it.
Using the mathematical formula we have calculated minimum possible distance for every demand and supply point and assigned it as a cost for it.

In [9]:
from math import sin, cos, sqrt, atan2
R = 6373.0
cost_matrix=np.zeros([13,148])
for i in range (13):
    loc = supply.loc[i,'Location']
    lat1=location.loc[loc,'LAT']
    lon1 = location.loc[loc,'LONG']
    for j in range (148):
        loc2 = demand.loc[j,'Location']
        lat2=location.loc[loc2,'LAT']
        lon2 = location.loc[loc2,'LONG']
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        distance = R * c
        cost_matrix[i][j]=distance

In [10]:
cost_matrix.shape

(13, 148)

In [11]:
cost_df=pd.DataFrame(cost_matrix)

In [12]:
cost_df

,0,1,2,3,4,5,6,7,8,9,...,138,139,140,141,142,143,144,145,146,147
0,7176.303947,16277.681570,2210.463543,4831.772243,15268.493586,6932.349178,5139.047439,16505.254143,6895.807971,7607.940159,...,7963.719768,10047.641878,17955.557784,16024.164183,14255.686734,14839.476578,16752.581308,8742.509683,17613.555430,14829.886421
1,14812.930047,1642.507130,16166.638206,16650.047526,4237.240711,13189.304743,14916.871916,5728.517141,15166.532158,14429.113970,...,14350.619751,9448.366273,3860.291434,1608.755100,6191.641511,3166.521338,5288.475907,13669.987171,4267.878354,7403.158601
2,15141.314073,7139.918317,15921.551724,16680.725592,1660.593315,18893.674823,19321.551401,5429.143275,14985.221415,14801.971637,...,13352.687195,15222.684719,4359.368194,5986.158442,11544.026461,4618.641162,8488.064096,11835.111454,7992.081177,6005.589734
3,2992.330867,14215.720547,4734.693259,1641.525465,18927.965069,2448.300849,1742.341108,13426.236343,3109.821286,3370.195929,...,4757.357674,6246.635226,15039.268029,15529.593321,9043.442593,17224.043990,11158.678278,6265.720114,11911.154653,12467.055771
4,3798.636615,16244.094127,3378.377635,1291.170964,18285.264195,4341.342175,2716.683412,14488.110093,3621.128121,4251.283731,...,5067.570923,8155.496212,16464.106287,17331.485983,11360.296911,17882.696829,13184.590108,6315.244977,14050.284687,13070.826886
5,3381.147334,16119.292421,8528.084708,4702.958703,13546.605449,8292.702667,7541.561902,9398.845481,3055.432834,3290.597605,...,1919.992984,11596.501505,11399.678423,16577.842014,11608.663318,16603.121730,10133.054068,1453.404577,11154.207957,7793.735198
6,16353.390298,5194.653556,16340.678883,18546.241961,324.575268,17123.256546,18599.486470,5637.480790,16357.083184,15924.540201,...,14719.114182,13288.184830,3920.983450,4006.305485,10045.065420,2773.095715,7762.689834,13278.978151,7020.215337,6754.064702
7,7506.588813,14907.369705,835.850220,4982.997040,15732.014332,5980.949238,4320.432285,17865.951076,7318.173099,7959.308852,...,8656.517682,8685.872422,19414.228189,14961.864917,13069.024239,14307.442577,16417.708148,9687.873341,16902.188253,16194.348877
8,16574.354014,5317.232597,16055.531783,18610.482676,423.455861,17247.129869,18555.196481,5902.528175,16552.198699,16153.775287,...,14903.231146,13396.936087,4207.669422,4085.383473,10274.158436,2707.707957,8051.838056,13441.985962,7302.724472,6982.708875
9,9447.583212,13394.319085,2389.876776,6915.472966,14001.667123,7165.906990,5782.434107,18892.353729,9270.168828,9900.971506,...,10602.126833,8927.254321,17864.033176,13170.884507,13389.054441,12354.645153,17402.133179,11566.767468,17240.914847,17531.059516


In [84]:
#cost_df['name']=supply['Location']

In [85]:
#cost_df.set_index('name')

In [13]:
cost_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,138,139,140,141,142,143,144,145,146,147
0,7176.303947,16277.681570,2210.463543,4831.772243,15268.493586,6932.349178,5139.047439,16505.254143,6895.807971,7607.940159,...,7963.719768,10047.641878,17955.557784,16024.164183,14255.686734,14839.476578,16752.581308,8742.509683,17613.555430,14829.886421
1,14812.930047,1642.507130,16166.638206,16650.047526,4237.240711,13189.304743,14916.871916,5728.517141,15166.532158,14429.113970,...,14350.619751,9448.366273,3860.291434,1608.755100,6191.641511,3166.521338,5288.475907,13669.987171,4267.878354,7403.158601
2,15141.314073,7139.918317,15921.551724,16680.725592,1660.593315,18893.674823,19321.551401,5429.143275,14985.221415,14801.971637,...,13352.687195,15222.684719,4359.368194,5986.158442,11544.026461,4618.641162,8488.064096,11835.111454,7992.081177,6005.589734


In [14]:
cost_df1=cost_df.T

In [15]:
cost_df1['name2']=demand['Location']

In [16]:
cost_df2=cost_df1.T
cost_df2

,0,1,2,3,4,5,6,7,8,9,...,138,139,140,141,142,143,144,145,146,147
0,7176.3,16277.7,2210.46,4831.77,15268.5,6932.35,5139.05,16505.3,6895.81,7607.94,...,7963.72,10047.6,17955.6,16024.2,14255.7,14839.5,16752.6,8742.51,17613.6,14829.9
1,14812.9,1642.51,16166.6,16650,4237.24,13189.3,14916.9,5728.52,15166.5,14429.1,...,14350.6,9448.37,3860.29,1608.76,6191.64,3166.52,5288.48,13670,4267.88,7403.16
2,15141.3,7139.92,15921.6,16680.7,1660.59,18893.7,19321.6,5429.14,14985.2,14802,...,13352.7,15222.7,4359.37,5986.16,11544,4618.64,8488.06,11835.1,7992.08,6005.59
3,2992.33,14215.7,4734.69,1641.53,18928,2448.3,1742.34,13426.2,3109.82,3370.2,...,4757.36,6246.64,15039.3,15529.6,9043.44,17224,11158.7,6265.72,11911.2,12467.1
4,3798.64,16244.1,3378.38,1291.17,18285.3,4341.34,2716.68,14488.1,3621.13,4251.28,...,5067.57,8155.5,16464.1,17331.5,11360.3,17882.7,13184.6,6315.24,14050.3,13070.8
5,3381.15,16119.3,8528.08,4702.96,13546.6,8292.7,7541.56,9398.85,3055.43,3290.6,...,1919.99,11596.5,11399.7,16577.8,11608.7,16603.1,10133.1,1453.4,11154.2,7793.74
6,16353.4,5194.65,16340.7,18546.2,324.575,17123.3,18599.5,5637.48,16357.1,15924.5,...,14719.1,13288.2,3920.98,4006.31,10045.1,2773.1,7762.69,13279,7020.22,6754.06
7,7506.59,14907.4,835.85,4983,15732,5980.95,4320.43,17866,7318.17,7959.31,...,8656.52,8685.87,19414.2,14961.9,13069,14307.4,16417.7,9687.87,16902.2,16194.3
8,16574.4,5317.23,16055.5,18610.5,423.456,17247.1,18555.2,5902.53,16552.2,16153.8,...,14903.2,13396.9,4207.67,4085.38,10274.2,2707.71,8051.84,13442,7302.72,6982.71
9,9447.58,13394.3,2389.88,6915.47,14001.7,7165.91,5782.43,18892.4,9270.17,9900.97,...,10602.1,8927.25,17864,13170.9,13389.1,12354.6,17402.1,11566.8,17240.9,17531.1


In [17]:
#cost_df2['name2']

In [18]:
cost_df2.columns = cost_df2.loc['name2']

In [19]:
cost_df2.head(5)

name2,"Birmingham, AL","Mobile, AL","Montgomery, AL","Tuscaloosa, AL","Anchorage, AK","Fairbanks, AK","Juneau, AK","Flagstaff, AZ","Phoenix, AZ","Scottsdale, AZ",...,"Wheeling, WV","Green Bay, WI","Madison, WI","Milwaukee, WI","Oshkosh, WI","Casper, WY","Cheyenne, WY","Cody, WY","Evanston, WY","Laramie, WY"
0,7176.3,16277.7,2210.46,4831.77,15268.5,6932.35,5139.05,16505.3,6895.81,7607.94,...,7963.72,10047.6,17955.6,16024.2,14255.7,14839.5,16752.6,8742.51,17613.6,14829.9
1,14812.9,1642.51,16166.6,16650,4237.24,13189.3,14916.9,5728.52,15166.5,14429.1,...,14350.6,9448.37,3860.29,1608.76,6191.64,3166.52,5288.48,13670,4267.88,7403.16
2,15141.3,7139.92,15921.6,16680.7,1660.59,18893.7,19321.6,5429.14,14985.2,14802,...,13352.7,15222.7,4359.37,5986.16,11544,4618.64,8488.06,11835.1,7992.08,6005.59
3,2992.33,14215.7,4734.69,1641.53,18928,2448.3,1742.34,13426.2,3109.82,3370.2,...,4757.36,6246.64,15039.3,15529.6,9043.44,17224,11158.7,6265.72,11911.2,12467.1
4,3798.64,16244.1,3378.38,1291.17,18285.3,4341.34,2716.68,14488.1,3621.13,4251.28,...,5067.57,8155.5,16464.1,17331.5,11360.3,17882.7,13184.6,6315.24,14050.3,13070.8


In [24]:
cost_df2['name']=supply['Location']
cost_df=cost_df2.set_index('name')
cost_df

name2,"Birmingham, AL","Mobile, AL","Montgomery, AL","Tuscaloosa, AL","Anchorage, AK","Fairbanks, AK","Juneau, AK","Flagstaff, AZ","Phoenix, AZ","Scottsdale, AZ",...,"Wheeling, WV","Green Bay, WI","Madison, WI","Milwaukee, WI","Oshkosh, WI","Casper, WY","Cheyenne, WY","Cody, WY","Evanston, WY","Laramie, WY"
name,,,,,,,,,,,,,,,,,,,,,
"Tucson, AZ",7176.3,16277.7,2210.46,4831.77,15268.5,6932.35,5139.05,16505.3,6895.81,7607.94,...,7963.72,10047.6,17955.6,16024.2,14255.7,14839.5,16752.6,8742.51,17613.6,14829.9
"Fresno, CA",14812.9,1642.51,16166.6,16650,4237.24,13189.3,14916.9,5728.52,15166.5,14429.1,...,14350.6,9448.37,3860.29,1608.76,6191.64,3166.52,5288.48,13670,4267.88,7403.16
"Lansing, MI",15141.3,7139.92,15921.6,16680.7,1660.59,18893.7,19321.6,5429.14,14985.2,14802,...,13352.7,15222.7,4359.37,5986.16,11544,4618.64,8488.06,11835.1,7992.08,6005.59
"Bozeman, MT",2992.33,14215.7,4734.69,1641.53,18928,2448.3,1742.34,13426.2,3109.82,3370.2,...,4757.36,6246.64,15039.3,15529.6,9043.44,17224,11158.7,6265.72,11911.2,12467.1
"Reno, NV",3798.64,16244.1,3378.38,1291.17,18285.3,4341.34,2716.68,14488.1,3621.13,4251.28,...,5067.57,8155.5,16464.1,17331.5,11360.3,17882.7,13184.6,6315.24,14050.3,13070.8
"Trenton, NJ",3381.15,16119.3,8528.08,4702.96,13546.6,8292.7,7541.56,9398.85,3055.43,3290.6,...,1919.99,11596.5,11399.7,16577.8,11608.7,16603.1,10133.1,1453.4,11154.2,7793.74
"Greensboro, NC",16353.4,5194.65,16340.7,18546.2,324.575,17123.3,18599.5,5637.48,16357.1,15924.5,...,14719.1,13288.2,3920.98,4006.31,10045.1,2773.1,7762.69,13279,7020.22,6754.06
"Columbus, OH",7506.59,14907.4,835.85,4983,15732,5980.95,4320.43,17866,7318.17,7959.31,...,8656.52,8685.87,19414.2,14961.9,13069,14307.4,16417.7,9687.87,16902.2,16194.3
"Tulsa, OK",16574.4,5317.23,16055.5,18610.5,423.456,17247.1,18555.2,5902.53,16552.2,16153.8,...,14903.2,13396.9,4207.67,4085.38,10274.2,2707.71,8051.84,13442,7302.72,6982.71


In [29]:
#cost_df.drop(index='NaN',axis=0)
cost_df = cost_df.iloc[:12,:]
cost_df.head()

name2,"Birmingham, AL","Mobile, AL","Montgomery, AL","Tuscaloosa, AL","Anchorage, AK","Fairbanks, AK","Juneau, AK","Flagstaff, AZ","Phoenix, AZ","Scottsdale, AZ",...,"Wheeling, WV","Green Bay, WI","Madison, WI","Milwaukee, WI","Oshkosh, WI","Casper, WY","Cheyenne, WY","Cody, WY","Evanston, WY","Laramie, WY"
name,,,,,,,,,,,,,,,,,,,,,
"Tucson, AZ",7176.3,16277.7,2210.46,4831.77,15268.5,6932.35,5139.05,16505.3,6895.81,7607.94,...,7963.72,10047.6,17955.6,16024.2,14255.7,14839.5,16752.6,8742.51,17613.6,14829.9
"Fresno, CA",14812.9,1642.51,16166.6,16650,4237.24,13189.3,14916.9,5728.52,15166.5,14429.1,...,14350.6,9448.37,3860.29,1608.76,6191.64,3166.52,5288.48,13670,4267.88,7403.16
"Lansing, MI",15141.3,7139.92,15921.6,16680.7,1660.59,18893.7,19321.6,5429.14,14985.2,14802,...,13352.7,15222.7,4359.37,5986.16,11544,4618.64,8488.06,11835.1,7992.08,6005.59
"Bozeman, MT",2992.33,14215.7,4734.69,1641.53,18928,2448.3,1742.34,13426.2,3109.82,3370.2,...,4757.36,6246.64,15039.3,15529.6,9043.44,17224,11158.7,6265.72,11911.2,12467.1
"Reno, NV",3798.64,16244.1,3378.38,1291.17,18285.3,4341.34,2716.68,14488.1,3621.13,4251.28,...,5067.57,8155.5,16464.1,17331.5,11360.3,17882.7,13184.6,6315.24,14050.3,13070.8


## Calculating rowwise and columnwise sums.

In [31]:
#suppliers 
cost_df.sum(axis=1).sort_values(ascending=False)

name
Bristol, TN       1.531214e+06
Fresno, CA        1.530199e+06
Tulsa, OK         1.528918e+06
Greensboro, NC    1.527722e+06
Harrisburg, PA    1.507638e+06
Lansing, MI       1.506433e+06
Columbus, OH      1.477464e+06
Tucson, AZ        1.468509e+06
Bozeman, MT       1.430981e+06
Reno, NV          1.427841e+06
Florence, SC      1.419308e+06
Trenton, NJ       1.386610e+06
dtype: float64

In [33]:
#demand 
cost_df.sum(axis=0).sort_values(ascending=True)

name2
Paris, TX           113562.595166
Louisville, KY      114244.491944
Columbia, SC        114643.188792
Miami, FL           114819.478738
Montgomery, AL      114964.529356
                        ...      
Flagstaff, AZ       125527.993030
Evanston, WY        125662.293733
Madison, WI         125968.103620
Danbury, CT         127120.195552
Newport News, VA    127132.470656
Length: 148, dtype: float64

## Modifying supply datasets

In [40]:
supply=supply.drop(index=[11,1,8,6],axis=0)

In [43]:
supply.loc[13,"Location"]="Paris, TX"
supply.loc[14,"Location"]="Louisville, KY"
supply.loc[13:14,"Supply"]=5000

In [56]:
supply=supply.reset_index()

In [57]:
supply=supply.drop('index',axis=1)
supply

,Location,Supply
0,"Tucson, AZ",2000.0
1,"Lansing, MI",2600.0
2,"Bozeman, MT",600.0
3,"Reno, NV",2500.0
4,"Trenton, NJ",3500.0
5,"Columbus, OH",2200.0
6,"Harrisburg, PA",2500.0
7,"Florence, SC",2500.0
8,"Spokane, WA",3000.0
9,"Paris, TX",5000.0


### New cost matrix

In [58]:
from math import sin, cos, sqrt, atan2
R = 6373.0
new_cost_matrix=np.zeros([11,148])
for i in range (11):
    loc = supply.loc[i,'Location']
    lat1=location.loc[loc,'LAT']
    lon1 = location.loc[loc,'LONG']
    for j in range (148):
        loc2 = demand.loc[j,'Location']
        lat2=location.loc[loc2,'LAT']
        lon2 = location.loc[loc2,'LONG']
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        distance = R * c
        new_cost_matrix[i][j]=distance

In [59]:
new_cost_df=pd.DataFrame(new_cost_matrix)

In [60]:
new_cost_df

,0,1,2,3,4,5,6,7,8,9,...,138,139,140,141,142,143,144,145,146,147
0,7176.303947,16277.681570,2210.463543,4831.772243,15268.493586,6932.349178,5139.047439,16505.254143,6895.807971,7607.940159,...,7963.719768,10047.641878,17955.557784,16024.164183,14255.686734,14839.476578,16752.581308,8742.509683,17613.555430,14829.886421
1,15141.314073,7139.918317,15921.551724,16680.725592,1660.593315,18893.674823,19321.551401,5429.143275,14985.221415,14801.971637,...,13352.687195,15222.684719,4359.368194,5986.158442,11544.026461,4618.641162,8488.064096,11835.111454,7992.081177,6005.589734
2,2992.330867,14215.720547,4734.693259,1641.525465,18927.965069,2448.300849,1742.341108,13426.236343,3109.821286,3370.195929,...,4757.357674,6246.635226,15039.268029,15529.593321,9043.442593,17224.043990,11158.678278,6265.720114,11911.154653,12467.055771
3,3798.636615,16244.094127,3378.377635,1291.170964,18285.264195,4341.342175,2716.683412,14488.110093,3621.128121,4251.283731,...,5067.570923,8155.496212,16464.106287,17331.485983,11360.296911,17882.696829,13184.590108,6315.244977,14050.284687,13070.826886
4,3381.147334,16119.292421,8528.084708,4702.958703,13546.605449,8292.702667,7541.561902,9398.845481,3055.432834,3290.597605,...,1919.992984,11596.501505,11399.678423,16577.842014,11608.663318,16603.121730,10133.054068,1453.404577,11154.207957,7793.735198
5,7506.588813,14907.369705,835.850220,4982.997040,15732.014332,5980.949238,4320.432285,17865.951076,7318.173099,7959.308852,...,8656.517682,8685.872422,19414.228189,14961.864917,13069.024239,14307.442577,16417.708148,9687.873341,16902.188253,16194.348877
6,9447.583212,13394.319085,2389.876776,6915.472966,14001.667123,7165.906990,5782.434107,18892.353729,9270.168828,9900.971506,...,10602.126833,8927.254321,17864.033176,13170.884507,13389.054441,12354.645153,17402.133179,11566.767468,17240.914847,17531.059516
7,5411.663345,9560.635360,11527.434589,7560.970126,12104.029452,6772.004703,8041.651814,6991.515502,5780.183880,5045.055313,...,5446.631896,6759.834448,8237.278089,10734.292239,4533.580444,12569.185430,4371.945471,5803.132896,5262.042555,6830.032925
8,10968.854198,5292.154483,17182.979767,13376.879621,6253.978280,12021.149710,13667.607384,2711.683655,11238.325989,10532.240615,...,10179.892562,9655.201790,2582.193622,5806.849872,5173.067294,7072.987803,1577.527196,9493.640678,1220.356206,4136.245176
9,7229.119471,15029.144614,682.978312,4703.111363,16008.649685,5758.360728,4072.898021,17678.582238,7045.366882,7682.087723,...,8399.889040,8567.797367,19500.205403,15155.512020,12909.552414,14568.847739,16162.354602,9456.514354,16690.294545,16027.240004


In [61]:
new_cost_df1=new_cost_df.T

In [62]:
new_cost_df1['name2']=demand['Location']

In [63]:
new_cost_df2=new_cost_df1.T
new_cost_df2

,0,1,2,3,4,5,6,7,8,9,...,138,139,140,141,142,143,144,145,146,147
0,7176.3,16277.7,2210.46,4831.77,15268.5,6932.35,5139.05,16505.3,6895.81,7607.94,...,7963.72,10047.6,17955.6,16024.2,14255.7,14839.5,16752.6,8742.51,17613.6,14829.9
1,15141.3,7139.92,15921.6,16680.7,1660.59,18893.7,19321.6,5429.14,14985.2,14802,...,13352.7,15222.7,4359.37,5986.16,11544,4618.64,8488.06,11835.1,7992.08,6005.59
2,2992.33,14215.7,4734.69,1641.53,18928,2448.3,1742.34,13426.2,3109.82,3370.2,...,4757.36,6246.64,15039.3,15529.6,9043.44,17224,11158.7,6265.72,11911.2,12467.1
3,3798.64,16244.1,3378.38,1291.17,18285.3,4341.34,2716.68,14488.1,3621.13,4251.28,...,5067.57,8155.5,16464.1,17331.5,11360.3,17882.7,13184.6,6315.24,14050.3,13070.8
4,3381.15,16119.3,8528.08,4702.96,13546.6,8292.7,7541.56,9398.85,3055.43,3290.6,...,1919.99,11596.5,11399.7,16577.8,11608.7,16603.1,10133.1,1453.4,11154.2,7793.74
5,7506.59,14907.4,835.85,4983,15732,5980.95,4320.43,17866,7318.17,7959.31,...,8656.52,8685.87,19414.2,14961.9,13069,14307.4,16417.7,9687.87,16902.2,16194.3
6,9447.58,13394.3,2389.88,6915.47,14001.7,7165.91,5782.43,18892.4,9270.17,9900.97,...,10602.1,8927.25,17864,13170.9,13389.1,12354.6,17402.1,11566.8,17240.9,17531.1
7,5411.66,9560.64,11527.4,7560.97,12104,6772,8041.65,6991.52,5780.18,5045.06,...,5446.63,6759.83,8237.28,10734.3,4533.58,12569.2,4371.95,5803.13,5262.04,6830.03
8,10968.9,5292.15,17183,13376.9,6253.98,12021.1,13667.6,2711.68,11238.3,10532.2,...,10179.9,9655.2,2582.19,5806.85,5173.07,7072.99,1577.53,9493.64,1220.36,4136.25
9,7229.12,15029.1,682.978,4703.11,16008.6,5758.36,4072.9,17678.6,7045.37,7682.09,...,8399.89,8567.8,19500.2,15155.5,12909.6,14568.8,16162.4,9456.51,16690.3,16027.2


In [64]:
new_cost_df2.columns = new_cost_df2.loc['name2']

In [65]:
new_cost_df2.head(5)

name2,"Birmingham, AL","Mobile, AL","Montgomery, AL","Tuscaloosa, AL","Anchorage, AK","Fairbanks, AK","Juneau, AK","Flagstaff, AZ","Phoenix, AZ","Scottsdale, AZ",...,"Wheeling, WV","Green Bay, WI","Madison, WI","Milwaukee, WI","Oshkosh, WI","Casper, WY","Cheyenne, WY","Cody, WY","Evanston, WY","Laramie, WY"
0,7176.3,16277.7,2210.46,4831.77,15268.5,6932.35,5139.05,16505.3,6895.81,7607.94,...,7963.72,10047.6,17955.6,16024.2,14255.7,14839.5,16752.6,8742.51,17613.6,14829.9
1,15141.3,7139.92,15921.6,16680.7,1660.59,18893.7,19321.6,5429.14,14985.2,14802,...,13352.7,15222.7,4359.37,5986.16,11544,4618.64,8488.06,11835.1,7992.08,6005.59
2,2992.33,14215.7,4734.69,1641.53,18928,2448.3,1742.34,13426.2,3109.82,3370.2,...,4757.36,6246.64,15039.3,15529.6,9043.44,17224,11158.7,6265.72,11911.2,12467.1
3,3798.64,16244.1,3378.38,1291.17,18285.3,4341.34,2716.68,14488.1,3621.13,4251.28,...,5067.57,8155.5,16464.1,17331.5,11360.3,17882.7,13184.6,6315.24,14050.3,13070.8
4,3381.15,16119.3,8528.08,4702.96,13546.6,8292.7,7541.56,9398.85,3055.43,3290.6,...,1919.99,11596.5,11399.7,16577.8,11608.7,16603.1,10133.1,1453.4,11154.2,7793.74


In [66]:
new_cost_df2['name']=supply['Location']
new_cost_df=new_cost_df2.set_index('name')
new_cost_df

name2,"Birmingham, AL","Mobile, AL","Montgomery, AL","Tuscaloosa, AL","Anchorage, AK","Fairbanks, AK","Juneau, AK","Flagstaff, AZ","Phoenix, AZ","Scottsdale, AZ",...,"Wheeling, WV","Green Bay, WI","Madison, WI","Milwaukee, WI","Oshkosh, WI","Casper, WY","Cheyenne, WY","Cody, WY","Evanston, WY","Laramie, WY"
name,,,,,,,,,,,,,,,,,,,,,
"Tucson, AZ",7176.3,16277.7,2210.46,4831.77,15268.5,6932.35,5139.05,16505.3,6895.81,7607.94,...,7963.72,10047.6,17955.6,16024.2,14255.7,14839.5,16752.6,8742.51,17613.6,14829.9
"Lansing, MI",15141.3,7139.92,15921.6,16680.7,1660.59,18893.7,19321.6,5429.14,14985.2,14802,...,13352.7,15222.7,4359.37,5986.16,11544,4618.64,8488.06,11835.1,7992.08,6005.59
"Bozeman, MT",2992.33,14215.7,4734.69,1641.53,18928,2448.3,1742.34,13426.2,3109.82,3370.2,...,4757.36,6246.64,15039.3,15529.6,9043.44,17224,11158.7,6265.72,11911.2,12467.1
"Reno, NV",3798.64,16244.1,3378.38,1291.17,18285.3,4341.34,2716.68,14488.1,3621.13,4251.28,...,5067.57,8155.5,16464.1,17331.5,11360.3,17882.7,13184.6,6315.24,14050.3,13070.8
"Trenton, NJ",3381.15,16119.3,8528.08,4702.96,13546.6,8292.7,7541.56,9398.85,3055.43,3290.6,...,1919.99,11596.5,11399.7,16577.8,11608.7,16603.1,10133.1,1453.4,11154.2,7793.74
"Columbus, OH",7506.59,14907.4,835.85,4983,15732,5980.95,4320.43,17866,7318.17,7959.31,...,8656.52,8685.87,19414.2,14961.9,13069,14307.4,16417.7,9687.87,16902.2,16194.3
"Harrisburg, PA",9447.58,13394.3,2389.88,6915.47,14001.7,7165.91,5782.43,18892.4,9270.17,9900.97,...,10602.1,8927.25,17864,13170.9,13389.1,12354.6,17402.1,11566.8,17240.9,17531.1
"Florence, SC",5411.66,9560.64,11527.4,7560.97,12104,6772,8041.65,6991.52,5780.18,5045.06,...,5446.63,6759.83,8237.28,10734.3,4533.58,12569.2,4371.95,5803.13,5262.04,6830.03
"Spokane, WA",10968.9,5292.15,17183,13376.9,6253.98,12021.1,13667.6,2711.68,11238.3,10532.2,...,10179.9,9655.2,2582.19,5806.85,5173.07,7072.99,1577.53,9493.64,1220.36,4136.25


In [67]:
new_cost_df = new_cost_df.iloc[:12,:]
new_cost_df.head()

name2,"Birmingham, AL","Mobile, AL","Montgomery, AL","Tuscaloosa, AL","Anchorage, AK","Fairbanks, AK","Juneau, AK","Flagstaff, AZ","Phoenix, AZ","Scottsdale, AZ",...,"Wheeling, WV","Green Bay, WI","Madison, WI","Milwaukee, WI","Oshkosh, WI","Casper, WY","Cheyenne, WY","Cody, WY","Evanston, WY","Laramie, WY"
name,,,,,,,,,,,,,,,,,,,,,
"Tucson, AZ",7176.3,16277.7,2210.46,4831.77,15268.5,6932.35,5139.05,16505.3,6895.81,7607.94,...,7963.72,10047.6,17955.6,16024.2,14255.7,14839.5,16752.6,8742.51,17613.6,14829.9
"Lansing, MI",15141.3,7139.92,15921.6,16680.7,1660.59,18893.7,19321.6,5429.14,14985.2,14802,...,13352.7,15222.7,4359.37,5986.16,11544,4618.64,8488.06,11835.1,7992.08,6005.59
"Bozeman, MT",2992.33,14215.7,4734.69,1641.53,18928,2448.3,1742.34,13426.2,3109.82,3370.2,...,4757.36,6246.64,15039.3,15529.6,9043.44,17224,11158.7,6265.72,11911.2,12467.1
"Reno, NV",3798.64,16244.1,3378.38,1291.17,18285.3,4341.34,2716.68,14488.1,3621.13,4251.28,...,5067.57,8155.5,16464.1,17331.5,11360.3,17882.7,13184.6,6315.24,14050.3,13070.8
"Trenton, NJ",3381.15,16119.3,8528.08,4702.96,13546.6,8292.7,7541.56,9398.85,3055.43,3290.6,...,1919.99,11596.5,11399.7,16577.8,11608.7,16603.1,10133.1,1453.4,11154.2,7793.74


## Minimization

In [68]:
import pandas as pd
!pip install pulp
from pulp import *
import numpy as np

In [69]:
Items =supply.Location.tolist()
Items

['Tucson, AZ',
 'Lansing, MI',
 'Bozeman, MT',
 'Reno, NV',
 'Trenton, NJ',
 'Columbus, OH',
 'Harrisburg, PA',
 'Florence, SC',
 'Spokane, WA',
 'Paris, TX',
 'Louisville, KY']

In [70]:
supply_dict = supply.set_index('Location')['Supply'].to_dict()
supply_dict

{'Tucson, AZ': 2000.0,
 'Lansing, MI': 2600.0,
 'Bozeman, MT': 600.0,
 'Reno, NV': 2500.0,
 'Trenton, NJ': 3500.0,
 'Columbus, OH': 2200.0,
 'Harrisburg, PA': 2500.0,
 'Florence, SC': 2500.0,
 'Spokane, WA': 3000.0,
 'Paris, TX': 5000.0,
 'Louisville, KY': 5000.0}

In [71]:
Items1 =demand.Location.tolist()
Items1

['Birmingham, AL',
 'Mobile, AL',
 'Montgomery, AL',
 'Tuscaloosa, AL',
 'Anchorage, AK',
 'Fairbanks, AK',
 'Juneau, AK',
 'Flagstaff, AZ',
 'Phoenix, AZ',
 'Scottsdale, AZ',
 'Winslow, AZ',
 'Fayetteville, AR',
 'Little Rock, AR',
 'Texarkana, AR',
 'Bakersfield, CA',
 'Los Angeles, CA',
 'Palm Springs, CA',
 'Sacramento, CA',
 'San Diego, CA',
 'San Francisco, CA',
 'Aspen, CO',
 'Denver, CO',
 'Danbury, CT',
 'New Haven, CT',
 'Wilmington, DE',
 'Fort Lauderdale, FL',
 'Gainesville, FL',
 'Jacksonville, FL',
 'Miami, FL',
 'Orlando, FL',
 'Panama City, FL',
 'Tallahassee, FL',
 'Tampa, FL',
 'Athens-Clarke County, GA',
 'Atlanta, GA',
 'Columbus, GA',
 'Savannah, GA',
 'Honolulu, HI',
 'Boise City, ID',
 'Pocatello, ID',
 'Chicago, IL',
 'Peoria, IL',
 'Bloomington, IN',
 'Indianapolis, IN',
 'Muncie, IN',
 'Des Moines, IA',
 'Topeka, KS',
 'Wichita, KS',
 'Louisville, KY',
 'Paducah, KY',
 'Baton Rouge, LA',
 'New Orleans, LA',
 'Shreveport, LA',
 'Bangor, ME',
 'Bar Harbor, ME',


In [72]:
demand_dict = demand.set_index('Location')['Demand'].to_dict()
demand_dict

{'Birmingham, AL': 200,
 'Mobile, AL': 100,
 'Montgomery, AL': 150,
 'Tuscaloosa, AL': 50,
 'Anchorage, AK': 50,
 'Fairbanks, AK': 100,
 'Juneau, AK': 200,
 'Flagstaff, AZ': 50,
 'Phoenix, AZ': 300,
 'Scottsdale, AZ': 100,
 'Winslow, AZ': 50,
 'Fayetteville, AR': 200,
 'Little Rock, AR': 300,
 'Texarkana, AR': 50,
 'Bakersfield, CA': 50,
 'Los Angeles, CA': 500,
 'Palm Springs, CA': 100,
 'Sacramento, CA': 200,
 'San Diego, CA': 400,
 'San Francisco, CA': 250,
 'Aspen, CO': 100,
 'Denver, CO': 250,
 'Danbury, CT': 50,
 'New Haven, CT': 100,
 'Wilmington, DE': 250,
 'Fort Lauderdale, FL': 150,
 'Gainesville, FL': 100,
 'Jacksonville, FL': 400,
 'Miami, FL': 500,
 'Orlando, FL': 450,
 'Panama City, FL': 50,
 'Tallahassee, FL': 150,
 'Tampa, FL': 200,
 'Athens-Clarke County, GA': 100,
 'Atlanta, GA': 500,
 'Columbus, GA': 150,
 'Savannah, GA': 200,
 'Honolulu, HI': 500,
 'Boise City, ID': 150,
 'Pocatello, ID': 50,
 'Chicago, IL': 450,
 'Peoria, IL': 150,
 'Bloomington, IN': 100,
 'Indian

In [73]:
# Setting up the Problem 
prob = LpProblem("Transportation", LpMinimize)

In [74]:
# Creates a list of tuples containing all the possible routes for transport
Routes = [(w,b) for w in Items for b in Items1]

In [75]:
route_vars = LpVariable.dicts("Route",(Items,Items1),0)

In [76]:
prob += lpSum([route_vars[w][b]*new_cost_df.loc[w,b] for (w,b) in Routes])

In [77]:
for w in Items:
    prob += lpSum([route_vars[w][b] for b in Items1]) <= supply_dict[w], "Sum of Products out of Warehouse %s"%w

# The demand minimum constraints are added to prob for each demand node (bar)
for b in Items1:
    prob += lpSum([route_vars[w][b] for w in Items]) >= demand_dict[b], "Sum of Products into Bars %s"%b

In [78]:
prob.solve()

1

In [79]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [80]:
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

Route_Bozeman,_MT_Denver,_CO = 250.0
Route_Bozeman,_MT_Indianapolis,_IN = 350.0
Route_Columbus,_OH_Tacoma,_WA = 100.0
Route_Florence,_SC_Chattanooga,_TN = 100.0
Route_Florence,_SC_Cheyenne,_WY = 150.0
Route_Florence,_SC_Duluth,_MN = 150.0
Route_Florence,_SC_Hoboken,_NJ = 150.0
Route_Florence,_SC_Laramie,_WY = 100.0
Route_Florence,_SC_Little_Rock,_AR = 300.0
Route_Florence,_SC_Muncie,_IN = 150.0
Route_Florence,_SC_New_York,_NY = 600.0
Route_Florence,_SC_Newark,_NJ = 250.0
Route_Florence,_SC_Olympia,_WA = 350.0
Route_Florence,_SC_Oshkosh,_WI = 50.0
Route_Florence,_SC_Portland,_ME = 150.0
Route_Harrisburg,_PA_Athens_Clarke_County,_GA = 100.0
Route_Harrisburg,_PA_Bar_Harbor,_ME = 50.0
Route_Harrisburg,_PA_Casper,_WY = 50.0
Route_Harrisburg,_PA_Elko,_NV = 100.0
Route_Harrisburg,_PA_Fargo,_ND = 50.0
Route_Harrisburg,_PA_Lincoln,_NE = 250.0
Route_Harrisburg,_PA_Memphis,_TN = 250.0
Route_Harrisburg,_PA_Miami,_FL = 500.0
Route_Harrisburg,_PA_Omaha,_NE = 300.0
Route_Harrisburg,_PA_Peoria,_IL = 1

In [81]:
print("Total transportation cost = ", value(prob.objective))

Total transportation cost =  131056784.63533074
